In [1]:
import os
from datetime import datetime, timedelta
from random import randint
from pytz import UTC

In [2]:
HEADER = """BEGIN:VCALENDAR
VERSION:2.0
PRODID:-//sme//ferien//EN
ATTENDEE:MAILTO:do-not@send-email.xyz
X-WR-CALNAME:Ferien2023"""

SUMMARY = """BEGIN:VEVENT
SUMMARY:{summary:s}
DTSTART;VALUE=DATE:{year:04d}{month:02d}{day:02d}
UID:{uid:s}@Ferien2023
DESCRIPTION:{description:s}
DTSTAMP;VALUE=DATE-TIME:{modified:s}
LAST-MODIFIED;VALUE=DATE-TIME:{modified:s}
LOCATION:Irrenhaus
ORGANIZER;CN="Ferien2023":MAILTO:do-not@send-email.xyz
URL:https://github.com/s-m-e
END:VEVENT"""

FOOTER = "END:VCALENDAR"

KURZ = {
    'Rheinland-Pfalz': 'RP',
    'Sachsen': 'SN',
    'Thüringen': 'TH',
    'Schleswig-Holstein': 'SH',
    'Bayern': 'BY',
    'Saarland': 'SD',
    'Niedersachsen': 'NS',
    'Sachsen-Anhalt': 'SA',
    'Berlin': 'B',
    'Bremen': 'HB',
    'Nordrhein-Westfalen': 'NRW',
    'Hamburg': 'HH',
    'Hessen': 'HE',
    'Baden-Württemberg': 'BW',
    'Mecklenburg-Vorpommern': 'MV',
    'Brandenburg': 'BB',
}

In [3]:
def parse(lines: str, state: str) -> list[list[str]]:
    
    events = {}
    buffer = {}
    in_event = False
    
    for line in lines.split('\n'):
        if line.strip() == 'BEGIN:VEVENT' and not in_event:
            in_event = True
            continue
        if line.strip() == 'END:VEVENT' and in_event:
            in_event = False
            summary = buffer.pop('SUMMARY')
            if summary in events.keys():
                if summary == 'Herbstferien':
                    summary = 'Herbstferien (2)'
                elif summary == 'Pfingstferien':
                    summary = 'Pfingstferien (2)'
                elif summary == 'Osterferien':
                    summary = 'Osterferien (2)'
                else:
                    raise ValueError(summary, events, state)
            events[summary] = buffer
            buffer = {}
            continue
        if in_event:
            if not any(line.startswith(prefix) for prefix in ('SUMMARY', 'DTSTART', 'DTEND')):
                continue
            if line.startswith('SUMMARY'):
                buffer['SUMMARY'] = line.split(':')[1].split(' 2023 ')[0]
                continue
            date = line.split(':')[1]
            buffer[line.split(';')[0]] = datetime(int(date[:4]), int(date[4:6]), int(date[6:]))

    if len(buffer) != 0:
        raise ValueError('buffer not empty')

    return events

In [4]:
def parse_all():
    
    fns = {
        fn.split('_')[1]: fn
        for fn in os.listdir()
        if 'ics' in fn
    }
    
    events_all = {}
    for state, short in KURZ.items():
        state = state.lower().replace('ö', 'oe').replace('ü', 'ue').replace('ä', 'ae')
        with open(fns[state], mode = 'r', encoding = 'utf-8') as f:
            raw = f.read()
        events_all[short] = parse(raw, state)
        
    events_sorted = {}
    for state, events in events_all.items():
        for name, dates in events.items():
            events_sorted[f'{name:s}/{state:s}'] = dates
    
    events_by_date = {}
    for days in range(0, 365):
        date = datetime(2023, 1, 1) + timedelta(days = days)
        buffer = []
        for name, dates in events_sorted.items():
            if dates['DTSTART'] <= date and date < dates['DTEND']:
                buffer.append(name)
        if len(buffer) == 0:
            continue
        events = {}
        for item in buffer:
            name, state = item.split('/')
            if name.endswith(' (2)'):
                name = name[:-4]
            if name not in events.keys():
                events[name] = []
            events[name].append(state)
        assert len(events) == 1
        event = list(events.keys())[0] + ' (' + ','.join(sorted(list(events.values())[0])) + ')'
        events_by_date[date] = event

    return events_by_date

In [5]:
def write_ical(fn: str, events: dict):
    
    n = datetime.now(tz = UTC)
    now = f'{n.year:04d}{n.month:02d}{n.day:02d}T{n.hour:02d}{n.minute:02d}{n.second:02d}Z'
    
    with open(fn, mode = 'w', encoding = 'utf-8') as f:
        f.write(HEADER + '\n')
        for date, name in events.items():
            f.write(SUMMARY.format(
                summary = name,
                year = date.year,
                month = date.month,
                day = date.day,
                uid = f'{randint(0, 2 ** 20):05x}',
                description = "Ferien",
                modified = now,
            ) + '\n')
        f.write(FOOTER)

In [6]:
events_by_date = parse_all()
write_ical('ferien_2023_2024.ical', events_by_date)

ValueError: ('Herbstferien 2024 Schleswig-Holstein', {'Osterferien 2024 Schleswig-Holstein': {'DTSTART': datetime.datetime(2024, 4, 2, 0, 0), 'DTEND': datetime.datetime(2024, 4, 20, 0, 0)}, 'Pfingstferien 2024 Schleswig-Holstein': {'DTSTART': datetime.datetime(2024, 5, 10, 0, 0), 'DTEND': datetime.datetime(2024, 5, 12, 0, 0)}, 'Sommerferien 2024 Schleswig-Holstein': {'DTSTART': datetime.datetime(2024, 7, 22, 0, 0), 'DTEND': datetime.datetime(2024, 9, 1, 0, 0)}, 'Herbstferien 2024 Schleswig-Holstein': {'DTSTART': datetime.datetime(2024, 10, 21, 0, 0), 'DTEND': datetime.datetime(2024, 11, 2, 0, 0)}}, 'schleswig-holstein')